## File Search with Programmatic Prompting

In [3]:
import os
import litellm
import json
import subprocess
from typing import List, Dict

os.environ['LM_STUDIO_API_BASE'] = "http://localhost:8090/v1"
os.environ['LM_STUDIO_API_KEY'] = "lm_studio"

In [80]:
def generate_response(messages: List[Dict]) -> str:
    """Call LLM to get response"""
    response = litellm.completion(
        model="lm_studio/meta-llama-3.1-8b-instruct",
        messages=messages,
        max_tokens=1024
    )
    return response.choices[0].message.content

agent_init = [
    {"role": "system", "content": """You are an intelligent AI agent that is capable of performing tasks with a given set of tools
    The tools at your disposal are as follows:
    1. list_files() : action=list files, args="None"
    2. change_dir(directory) : action=change directory, args=directory
    3. read_file(file_name) : action=read file, args=file_name
    4. terminate() : action=terminate, args="None"
    Every response must contain one action
    Every response should be in json format: {"action": tool_name, "args": tool_args}"}
    Once the task is complete, use the terminate action"""}
]

In [33]:
messages = agent_init + [{"role": "user", "content": "List files in the current directory"}]
response = generate_response(messages)
print(response)

{"action": "list_files", "args": "None"}


In [34]:
def parse_agent_response_for_action(response: str):
    lines = response.split("\n")
    for line in lines:
        if "{\"action\"" in line:
            agent_action = json.loads(line)
            return agent_action

agent_action = parse_agent_response_for_action(response)
print("Agent action:", agent_action)

Agent action: {'action': 'list_files', 'args': 'None'}


In [83]:
def execute_action(agent_action, directory):
    if "action" not in agent_action or "args" not in agent_action:
        return {"result": "Use the provided JSON format to perform an action"}
    tool = agent_action["action"]
    arg = agent_action["args"]
    if tool == "list_files":
        return {"result": subprocess.check_output("dir" + (" " + directory if directory is not None else ""), shell=True, text=True)}
    elif tool == "change_dir":
        new_dir = subprocess.check_output("cd " + arg + " && cd", shell=True, text=True).strip("\n")
        return {"result": "Changed directory to " + arg, "directory": new_dir}
    elif tool == "read_file":
        file_content = subprocess.check_output("type " + (directory + "\\" if directory is not None else "") + arg, shell=True, text=True)
        print(file_content)
        return {"result": "File read!"}   
    elif tool == "terminate":
        return {"result": "terminate"}
    else:
        return {"result": "Invalid action"}
        
execute_action({"action": "terminate", "args": ""}, None)

{'result': 'terminate'}

In [86]:
user_instruction = "Read the contents of files in the directory data"
print("User:", user_instruction)

memory = agent_init + [{"role": "user", "content": user_instruction}]
directory = None
while True:
    print("Agent: Thinking.... directory=", directory)

    response = generate_response(memory)
    agent_action = parse_agent_response_for_action(response)
    print("Executing action ", agent_action)
    output = execute_action(agent_action, directory)

    if "directory" in output.keys():
        directory = output["directory"]
    if output["result"] == "terminate":
        print("Agent: I'm done!")
        break
    
    memory += [{"role": "assistant", "content": response}, {"role": "user", "content": output["result"]}]
    print("-"*100)

User: Read the contents of files in the directory data
Agent: Thinking.... directory= None
Executing action  {'action': 'list_files', 'args': 'None'}
----------------------------------------------------------------------------------------------------
Agent: Thinking.... directory= None
Executing action  {'action': 'list_files', 'args': 'None'}
----------------------------------------------------------------------------------------------------
Agent: Thinking.... directory= None
Executing action  {'action': 'list_files', 'args': 'None'}
----------------------------------------------------------------------------------------------------
Agent: Thinking.... directory= None
Executing action  {'action': 'change_dir', 'args': 'data'}
----------------------------------------------------------------------------------------------------
Agent: Thinking.... directory= C:\Work\main\Courses\AgenticAIWithPython\Module1\LLMAgents\data
Executing action  {'action': 'list_files', 'args': 'None'}
-------